In [25]:
import geopandas as gpd
import pandas as pd
import os
from shapely.geometry import Polygon, LineString, MultiLineString, Point, GeometryCollection
pd.options.display.max_columns = 100

In [26]:
#Assigning Path
path_road = r'Z:\Masterfile\1. Shapefile\5. Road\1. All Road'
path_poi = r'Z:\Masterfile\1. Shapefile\4. POI'
path_zmp = r'Z:\000. Processing\REGION I\ZMP'

In [27]:
#Preparing and Reading Files
poi = gpd.read_file(os.path.join(path_poi, 'poi_priority.gpkg'))
road = gpd.read_file(os.path.join(path_road, 'Road_With_Characteristics.gpkg'))
zmp = gpd.read_file(os.path.join(path_zmp, 'ID_ZMP_MASTERFILE_REGION_I.shp'))

In [28]:
#Changing Prjection and creating road length
zmp = zmp.to_crs(epsg = 4326)
# road = road.to_crs(epsg = 32748)
# road['length'] = road.length
# road = road.to_crs(epsg = 4326)

In [29]:
# #Changing Projection
# zmp = zmp.to_crs(epsg = 4326)
# #Join zmp with road
# zmp_road = gpd.sjoin(road, zmp, how = 'right', predicate = 'intersects')
# #Creating rank
# zmp_road['rank'] = zmp_road.groupby(['ZMP_CODE'])['length'].rank(ascending = False)
# #Applying ZMP Category
# def zmp_cat(a):
#     if a=='>5 m':
#         return 'ZMP Commercial'
#     else:
#         return 'ZMP Non Commercial'
# zmp_road['zmp_category'] = zmp_road['Perkiraan Luas'].apply(lambda x: zmp_cat(x))
# zmp_road = zmp_road[['ZMP_CODE','zmp_category']]

# ZMP With POI Class

In [30]:
#Merging ZMP with POI
zmp_join = gpd.sjoin(poi, zmp, how = 'right', predicate='intersects')

In [31]:
# Create ZMP Summary with POI Class
zmp_dissolve = zmp_join.pivot_table(index = 'ZMP_CODE', values = 'Sub_Class', columns='POI_Class', aggfunc='count').reset_index().fillna(0)
zmp_dissolve

POI_Class,ZMP_CODE,CVS,Gov,Health Center,Religion,School,Transportation
0,3171010_005,1.0,0.0,1.0,0.0,1.0,0.0
1,3171010_008,0.0,1.0,0.0,1.0,0.0,0.0
2,3171010_010,1.0,0.0,0.0,0.0,0.0,0.0
3,3171010_012,0.0,0.0,1.0,0.0,0.0,0.0
4,3171010_017,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
3548,3674070_014,0.0,0.0,1.0,0.0,0.0,0.0
3549,3674070_015,0.0,0.0,0.0,0.0,1.0,0.0
3550,3674070_016,0.0,0.0,0.0,0.0,1.0,0.0
3551,3674070_018,0.0,1.0,0.0,0.0,0.0,0.0


In [32]:
#Define zmp category function
def category(a):
    if a['Religion'] >=1:
        return 'Residential'
    elif a['School']>=1:
        return 'School'
    elif a['CVS']>=1:
        return 'Commercial'
    elif a['Gov']>=1:
        return 'Goverment'
    elif a['Health Center']>=1:
        return 'Health Center'
    elif a['Transportation']>=1:
        return 'Transportation'
    else:
        return 'Commercial'

In [33]:
zmp_dissolve['zmp_category'] = zmp_dissolve[['CVS', 'Gov', 'Health Center', 'Religion', 'School',
       'Transportation']].apply(lambda x: category(x), axis = 1)

In [ ]:
zmp

In [34]:
zmp_dissolve.head()

POI_Class,ZMP_CODE,CVS,Gov,Health Center,Religion,School,Transportation,zmp_category
0,3171010_005,1.0,0.0,1.0,0.0,1.0,0.0,School
1,3171010_008,0.0,1.0,0.0,1.0,0.0,0.0,Residential
2,3171010_010,1.0,0.0,0.0,0.0,0.0,0.0,Commercial
3,3171010_012,0.0,0.0,1.0,0.0,0.0,0.0,Health Center
4,3171010_017,1.0,0.0,0.0,0.0,0.0,0.0,Commercial


In [41]:
zmp_dissolve[zmp_dissolve.zmp_category.isnull()]

POI_Class,ZMP_CODE,CVS,Gov,Health Center,Religion,School,Transportation,zmp_category


In [35]:
#Finalizing ZMP
zmp_final = zmp.merge(zmp_dissolve, how ='left', on = 'ZMP_CODE')
zmp_final = zmp_final[['PROVINCE', 'KECAMATAN', 'CITY', 'ZMP_CODE', 'ZMP_NAME','zmp_category','geometry']]
zmp_final = zmp_final.fillna('Commercial')
zmp_final.to_file(r'Z:\Masterfile\1. Shapefile\8. ZMP\zmp_masterfile.gpkg')